In [7]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/TeA/project/datasets/FINAL/DATA12"
!pwd
!ls -la

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/TeA/project/datasets/FINAL/DATA12
/content/drive/My Drive/TeA/project/datasets/FINAL/DATA12
total 28
drwx------ 2 root root 4096 Jul 13 08:21 10
drwx------ 2 root root 4096 Jul 13 08:21 20
drwx------ 2 root root 4096 Jul 13 08:21 30
drwx------ 2 root root 4096 Jul 13 08:22 40
drwx------ 2 root root 4096 Jul 13 08:22 50
drwx------ 2 root root 4096 Aug 18 14:43 data
drwx------ 3 root root 4096 Aug 21 08:04 datasets


In [8]:
#input file/dir = /content/drive/My Drive/TeA/project/dataset/datasets-test-v7/new_traffic1.csv

import sys
import subprocess
import pkg_resources
import os
import time
import platform
import datetime
import numpy as np
import csv
import click
from IPython.display import HTML, display

h = int(0)
m = int(0)
s = int(0)
d = int(0)

def addSecs(tm, secs):
    fulldate = datetime.datetime(100, 1, 1, tm.hour, tm.minute, tm.second)
    fulldate = fulldate + datetime.timedelta(seconds=secs)
    return fulldate.time()

def timer():
    global h
    global m
    global s
    global d
    
    if(s < 59):
        s += 1
    elif(s == 59):
        s = 0
        if(m < 59):
            m += 1
        elif(m == 59):
            m = 0
            if(h < 24):
                h += 1
            elif(h == 24):
                h = 0
                d += 1
    
    if(d<10):
        strD = str(0)+str(d)
    else:
        strD = str(d)

    if(h<10):
        strH = str(0)+str(h)
    else:
        strH = str(h)
            
    if(m<10):
        strM = str(0)+str(m)
    else:
        strM = str(m)
    
    if(s<10):
        strS = str(0)+str(s)
    else:
        strS = str(s)
    
    return strH + ":" + strM + ":" + strS

def progress(value, max):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

def is_ascii(s):
    return all(ord(c) < 128 for c in s)

def save(data_arr, filename, extract_folder): 
    if platform.system() == "Windows":
        parent_dataset_directory = os.getcwd() + "\datasets"
        child_dataset_directory = parent_dataset_directory + "\\" + extract_folder
        if not os.path.exists(parent_dataset_directory): os.mkdir(parent_dataset_directory)
        if not os.path.exists(child_dataset_directory): os.mkdir(child_dataset_directory)
        np.save(child_dataset_directory + "\\" + filename, np.array(data_arr))
    else:
        parent_dataset_directory = os.getcwd() + "/datasets"
        child_dataset_directory = parent_dataset_directory + "/" + extract_folder
        if not os.path.exists(parent_dataset_directory): os.mkdir(parent_dataset_directory)
        if not os.path.exists(child_dataset_directory): os.mkdir(child_dataset_directory)
        np.save(child_dataset_directory + "/" + filename, np.array(data_arr))
    
    print(f'Saved to => {child_dataset_directory}')

def extract_data(get_each_row_data, rowshelp):
    #Set zero-value for the rest of index array
    zero_val = int(0)
    val_255 = int(255)
    actual_img = (32**2) * 3
    data_text_ascii = ""
    info = ""
    
    #used parameters
    #['Source IP', 'Source Port', 'Destination IP', 'Destination Port', 'Protocol', 'Packet Length', 'Data Length', 'Data', 'Data Text', 'Info']
    
    #Source IP
    if get_each_row_data[0] != "":
        src_ip = np.fromstring(str(get_each_row_data[0]), dtype=np.uint8)
    else: 
        src_ip = zero_val
    data_arr = np.array([src_ip])

    #Source Port
    if get_each_row_data[1] != "":
        if int(get_each_row_data[1]) <= 255:
            #data_arr[0,1] = int(get_each_row_data[1])
            src_port = int(get_each_row_data[1])
        else: 
            #data_arr[0,1] = 255
            src_port = val_255
    else: 
        #data_arr[0,1] = 0
        src_port = zero_val
    data_arr = np.append(data_arr, src_port)
    
    #Destination IP
    if get_each_row_data[2] != "":
        dst_ip = np.fromstring(str(get_each_row_data[2]), dtype=np.uint8)
    else: 
        dst_ip = zero_val
    data_arr = np.append(data_arr, dst_ip)

    #Destination Port
    if get_each_row_data[3] != "":
        if int(get_each_row_data[3]) <= 255:
            #data_arr[0,1] = int(get_each_row_data[1])
            dst_port = int(get_each_row_data[3])
        else: 
            #data_arr[0,1] = 255
            dst_port = val_255
    else: 
        #data_arr[0,1] = 0
        dst_port = zero_val
    data_arr = np.append(data_arr, dst_port)

    #Protocol
    if get_each_row_data[4] != "":
        #convert_str = np.fromstring(str(get_each_row_data[2]), dtype=np.uint8)
        #data_arr[0,2] = convert_str
        protocol = np.fromstring(str(get_each_row_data[4]), dtype=np.uint8)
    else: 
        #data_arr[0,2] = 0
        protocol = zero_val
    data_arr = np.append(data_arr, protocol)

    #Packet Length
    if get_each_row_data[5] != "":
        if int(get_each_row_data[5]) <= 255:
            #data_arr[1,0] = int(get_each_row_data[3])
            packet_length = int(get_each_row_data[5])
        else: 
            #data_arr[1,0] = 255
            packet_length = val_255
    else: 
        #data_arr[1,0] = 0
        packet_length = zero_val
    data_arr = np.append(data_arr, packet_length)

    #Data Length
    if get_each_row_data[6] != "": #data_arr[1,1] = 255
        if int(get_each_row_data[6]) <= 255:
            #data_arr[1,1] = int(get_each_row_data[4])
            data_length = int(get_each_row_data[6])
        else: 
            #data_arr[1,1] = 255
            data_length = val_255
    else: 
        #data_arr[1,1] = 0
        data_length = zero_val
    data_arr = np.append(data_arr, data_length)

    #Data
    #Data Text
    if get_each_row_data[7] != "": 
        #data_arr[1,2] = 255
        data = val_255
        temp_data = str(get_each_row_data[8])
        for c in temp_data:
            if is_ascii(c): data_text_ascii += c
        data_text = np.fromstring(str(data_text_ascii), dtype=np.uint8)
    else: 
        #data_arr[1,2] = 0
        data = zero_val
        data_text = zero_val
    data_arr = np.append(data_arr, data)
    data_arr = np.append(data_arr, data_text)

    #Info
    if get_each_row_data[9] != "":
        #convert_str = np.fromstring(str(get_each_row_data[6]), dtype=np.uint8)
        #data_arr[2,0] = convert_str
        info = np.fromstring(str(get_each_row_data[9]), dtype=np.uint8)
    else: 
        #data_arr[2,0] = 0
        info = zero_val
    data_arr = np.append(data_arr, info)

    #Label set
    if get_each_row_data[4] != "" and \
        get_each_row_data[5] != "" and \
        get_each_row_data[9] != "":
        #ICMP
        if str(get_each_row_data[4]) == "ICMP" and \
            int(get_each_row_data[5]) >= 500:
            label_set = 1
        #SYN
        elif "[SYN]" in str(get_each_row_data[9]):
            label_set = 1
        #UDP
        elif str(rowshelp[11]) == "Destination unreachable (Port unreachable)":
            if str(get_each_row_data[4]) == "UDP":
                label_set = 1
            else: label_set = 0
        else: 
            label_set = 0
    else: 
        label_set = 0

    #a = "true" if get_each_row_data[4] == "" else "false"
    #data_arr = np.array([src_port, dst_port, protocol, packet_length, data_length, data, info]*175)

    #print(data_arr)
    #print(data_arr.shape)
    #print(len(data_arr))
    #input("Press Enter to exit...")
    #sys.exit()
    
    temp_arr = data_arr
    temp_loop = int(actual_img/len(temp_arr))
    #print(temp_loot)
    if int(actual_img%len(temp_arr)) == 0:
        for i in range(temp_loop-1): 
            data_arr = np.append(data_arr, temp_arr)
        data_arr = np.append(data_arr, label_set)
        return data_arr
    else:
        for i in range(temp_loop-1):
            data_arr = np.append(data_arr, temp_arr)
        for i in range(int(actual_img-len(data_arr))): 
            data_arr = np.append(data_arr, zero_val)
        data_arr = np.append(data_arr, label_set)
        return data_arr

def collectData(fileCSV, filename, extract_folder):
    #data_arr = np.zeros((0,1226), dtype=int)

    with open(fileCSV, mode='rt') as csv_file:
        csv_read = csv.reader(csv_file)
        rowsHelper=list(csv_read)

    with open(fileCSV, mode='rt') as csv_file:
        csv_reader = csv.reader(csv_file)
        print("counting data total..")
        row_count = sum(1 for row in csv_reader)
        if row_count < 1000: 
            print("too little bit of data, try to get more captured data")
            print("exitting..")
            sys.exit()
        #elif len(rowsHelper == row_count): print("data ok.")
        #row_count => count all data in csv file with header
        #row_count-1 => count all data in csv file without header
        else: print(f'{row_count-1} data')

    
    out = display(progress(0, row_count), display_id=True)
    with open(fileCSV, mode='rt') as csv_file:
        csv_reader = csv.reader(csv_file)
        temp_arr = np.zeros((0,3073), dtype=int)
        data_arr = temp_arr.astype(int)
        
        line_count = 0
        get_val = []
        get_col_name = []
        get_each_row_data = []
        init = datetime.datetime.now().time()
        now = addSecs(init, 0)
        later = addSecs(init, 1)
        for row in csv_reader:
            init = datetime.datetime.now().time()
            if addSecs(init, 0) == later:
                later = addSecs(init, 1)
                sys.stdout.write("\r")
                sys.stdout.write("Timer : {:}".format(timer()))
                sys.stdout.flush()

            if line_count == 0:
                get_val.extend(row)
                get_col_name = get_val[2:]
                get_col_name = []
                get_val = []
                line_count += 1
            else:
                get_val.extend(row)
                if line_count + 1 < row_count: rowHelp = rowsHelper[line_count + 1]
                get_each_row_data = get_val[2:]
                return_data_arr = extract_data(get_each_row_data, rowHelp)
                data_arr = np.vstack((data_arr, return_data_arr))
                get_each_row_data = []
                get_val = []
                line_count += 1
            out.update(progress(line_count, row_count))
        print()
        print(f'Processed {line_count-1} lines data.')
        print("saving file...")
        save(data_arr, filename, extract_folder)

def getFile(file_path, extract_folder):
    errors = []
    error = False
    file = os.path.split(file_path)[1]
    print("File selected : " + file)
    try: 
        print("Extracting data : " + file)
        start = datetime.datetime.now()
        TimeStart = addSecs(start.time(), 0)
        collectData(file_path, file, extract_folder)
    except (Exception, IndexError) as e: 
        error = True
        print("Error while extracting data : " + file)
        print("Error on line {}".format(sys.exc_info()[-1].tb_lineno))
        print("Error msg : ", e)
        print("exitting..")
        sys.exit()
    finally: 
        if not error:
            end = datetime.datetime.now()
            TimeEnd = addSecs(end.time(), 0)
            print("Time Start : {}".format(TimeStart))
            print('Time taken to extracting data : {}'.format(end-start))
            print("Time End : {}".format(TimeEnd))
            print("data extracted. DONE.")

def getFiles(file_dir, extract_folder):
    errors = []
    error = False
    print("Folder selected : " + file_dir)
    files = os.listdir(file_dir)
    for file in files:
        h = int(0)
        m = int(0)
        s = int(0)
        d = int(0)
        try:
            print("Extracting data : " + file)
            start = datetime.datetime.now()
            TimeStart = addSecs(start.time(), 0)
            collectData(file_dir+"/"+file, file, extract_folder)
        except (Exception, IndexError) as e:
            print("Error while extracting data : " + file)
            errors += file
            continue
        finally:
            if len(errors) == len(files):
                print("all data can't be extract..")
                print("Error on line {}".format(sys.exc_info()[-1].tb_lineno))
                print("Error msg : ", e)
                print("exitting..")
                sys.exit()
            elif len(errors) != 0: print("some data can't be extract..")
            else:
                end = datetime.datetime.now()
                TimeEnd = addSecs(end.time(), 0)
                print("Time Start : {}".format(TimeStart))
                print('Time taken to extracting data : {}'.format(end-start))
                print("Time End : {}".format(TimeEnd))
                print("data extracted. DONE.")

def getOptions():
    while True:
        if platform.system() == "Windows": os.system("cls")
        else: os.system("clear")
        print(" ")
        sep="="*26
        print(f'{sep}')
        file_path = input("Input the directory : ")
        file_path = os.path.abspath(file_path)
        extract_folder = input("Extracted folder name : ")
        print(f'Directory selected : {file_path}')
        print(f'Extracted folder name : ../{extract_folder}')
        if click.confirm('Double check the directory you just entered. continue?', default=True): break
    if platform.system() == "Windows": os.system("cls")
    else: os.system("clear")
    print(" ")
    print("Data Normalization")
    print("==================")
    if os.path.isfile(file_path): getFile(file_path, extract_folder)
    elif os.path.isdir(file_path): getFiles(file_path, extract_folder)
    else:
        print("incorrect directory folder or file. Now extiting...")
        sys.exit()

def main():
    try:
        print("starting...")
        time.sleep(1)
        getOptions()
    except KeyboardInterrupt:
        print("\nProgram interrupted, ctrl-C pressed. Now exiting....")
        time.sleep(2)
        if platform.system() == "Windows": os.system("cls")
        else: os.system("clear")
        sys.exit()

if __name__ == "__main__":
    main()


starting...
 
Input the directory : /content/drive/My Drive/TeA/project/datalog/logwireshark.csv
Extracted folder name : data_extract
Directory selected : /content/drive/My Drive/TeA/project/datalog/logwireshark.csv
Extracted folder name : ../data_extract
Double check the directory you just entered. continue? [Y/n]: y
 
Data Normalization
File selected : logwireshark.csv
Extracting data : logwireshark.csv
counting data total..
4036 data


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:110: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:130: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:152: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:192: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:204: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


Timer : 00:01:23
Processed 4036 lines data.
saving file...
Saved to => /content/drive/My Drive/TeA/project/datasets/FINAL/DATA12/datasets/data_extract
Time Start : 08:15:16
Time taken to extracting data : 0:01:23.039908
Time End : 08:16:39
data extracted. DONE.
